In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split

# Defining the MobileFaceNet model with fine-tuning
class MobileFaceNet(nn.Module):
    def __init__(self, num_classes):
        super(MobileFaceNet, self).__init__()
        self.model = models.mobilenet_v2(weights=True)
        self.model.classifier[-1] = nn.Linear(self.model.classifier[-1].in_features, num_classes)

        # Adding attention layers for masked face recognition
        self.attention = nn.Sequential(
            nn.Conv2d(1280, 128, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.model.features(x)
        attention_map = self.attention(x)
        non_occlusion_map = 1.0 - attention_map
        x = x * non_occlusion_map
        x = x.mean([2, 3])
        x = x.view(x.size(0), -1)
        x = self.model.classifier(x)
        return x

    
# Setting hyperparameters
batch_size = 16
num_classes = 11
learning_rate = 0.0001
num_epochs = 10

# Defining data transformations
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [2]:
# Loading the dataset
dataset = datasets.ImageFolder(root=r'C:\Users\adars\OneDrive\Desktop\Self_dataset', transform=transform)

# Splitting the dataset into train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Creating data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Creating an instance of the MobileFaceNet model
model = MobileFaceNet(num_classes)

# Defining the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


C:\Users\adars\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Validation Accuracy: {val_accuracy:.2f}%")

Epoch 1/10 - Validation Accuracy: 0.00%
Epoch 2/10 - Validation Accuracy: 20.00%
Epoch 3/10 - Validation Accuracy: 40.00%
Epoch 4/10 - Validation Accuracy: 80.00%
Epoch 5/10 - Validation Accuracy: 100.00%
Epoch 6/10 - Validation Accuracy: 100.00%
Epoch 7/10 - Validation Accuracy: 100.00%
Epoch 8/10 - Validation Accuracy: 100.00%
Epoch 9/10 - Validation Accuracy: 100.00%
Epoch 10/10 - Validation Accuracy: 100.00%


In [5]:
# After training
model.eval()
total = 0
correct = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Accuracy on the test set: {test_accuracy:.2f}%")

Accuracy on the test set: 100.00%
